In [3]:
%pwd

'c:\\Users\\user\\Code\\detection'

### Visualize Raw Data

In [5]:
import k3d
import numpy as np

with open ("000013.bin", "rb") as f:
    scan = np.fromfile(f, dtype=np.float32)
    scan = scan.reshape((-1, 4))

plot = k3d.plot()
plot +=k3d.points(scan[:,:3], point_size=0.02, attribute=scan[:,3], color_map=k3d.colormaps.basic_color_maps.Jet)
plot.display()

Output()

### Pre process step

In [9]:
import numpy as np
from sklearn.cluster import DBSCAN

# Load the .bin file as a numpy array
lidar_data = np.fromfile('000001.bin', dtype=np.float32).reshape(-1, 4)

# Ground removal based on Z threshold
ground_threshold = -1.5
non_ground_points = lidar_data[lidar_data[:, 2] > ground_threshold]

# Apply DBSCAN clustering to identify noise
db = DBSCAN(eps=0.3, min_samples=10).fit(non_ground_points[:, :3])
labels = db.labels_

# Extract core samples (non-noise points)
core_samples_mask = np.zeros_like(labels, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
filtered_points = non_ground_points[core_samples_mask]

#Specifies the file path for output. A bin file with the specified name is generated under the filtered directory.
output_path = './filtered/filtered_points.bin'
filtered_points.tofile(output_path)

### Preprocessed data

In [10]:
import k3d
import numpy as np

with open ("filtered/filtered_points.bin", "rb") as f:
    scan = np.fromfile(f, dtype=np.float32)
    scan = scan.reshape((-1, 4))

plot = k3d.plot()
plot +=k3d.points(scan[:,:3], point_size=0.02, attribute=scan[:,3], color_map=k3d.colormaps.basic_color_maps.Jet)
plot.display()

Output()

### Install the mmdetection3d library from git.

The mmdetection3d library is a library that provides models used for object detection in 3D point clouds.

https://github.com/open-mmlab

In [11]:
# %%
!pip3 install openmim
!mim install mmcv-full==1.7.0

# %%
!mim install mmdet

!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -e .
%cd ..

# %%
!git clone https://github.com/open-mmlab/mmdetection3d.git
%cd mmdetection3d
!pip install -e .

# %%


Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
c:\Users\user\Code\detection\mmsegmentation


Cloning into 'mmsegmentation'...
Updating files:  60% (1035/1725)
Updating files:  61% (1053/1725)
Updating files:  62% (1070/1725)
Updating files:  63% (1087/1725)
Updating files:  64% (1104/1725)
Updating files:  65% (1122/1725)
Updating files:  66% (1139/1725)
Updating files:  67% (1156/1725)
Updating files:  68% (1173/1725)
Updating files:  69% (1191/1725)
Updating files:  70% (1208/1725)
Updating files:  71% (1225/1725)
Updating files:  72% (1242/1725)
Updating files:  73% (1260/1725)
Updating files:  74% (1277/1725)
Updating files:  75% (1294/1725)
Updating files:  76% (1311/1725)
Updating files:  77% (1329/1725)
Updating files:  78% (1346/1725)
Updating files:  79% (1363/1725)
Updating files:  80% (1380/1725)
Updating files:  81% (1398/1725)
Updating files:  82% (1415/1725)
Updating files:  83% (1432/1725)
Updating files:  84% (1449/1725)
Updating files:  85% (1467/1725)
Updating files:  86% (1484/1725)
Updating files:  87% (1501/1725)
Updating files:  88% (1518/1725)
Updating f

Obtaining file:///C:/Users/user/Code/detection/mmsegmentation
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: mmsegmentation
    Found existing installation: mmsegmentation 1.1.2
    Uninstalling mmsegmentation-1.1.2:
      Successfully uninstalled mmsegmentation-1.1.2
  Running setup.py develop for mmsegmentation
c:\Users\user\Code\detection
c:\Users\user\Code\detection\mmdetection3d


Cloning into 'mmdetection3d'...
Updating files:  73% (832/1129)
Updating files:  74% (836/1129)
Updating files:  75% (847/1129)
Updating files:  76% (859/1129)
Updating files:  77% (870/1129)
Updating files:  78% (881/1129)
Updating files:  79% (892/1129)
Updating files:  80% (904/1129)
Updating files:  81% (915/1129)
Updating files:  82% (926/1129)
Updating files:  83% (938/1129)
Updating files:  84% (949/1129)
Updating files:  85% (960/1129)
Updating files:  86% (971/1129)
Updating files:  87% (983/1129)
Updating files:  88% (994/1129)
Updating files:  89% (1005/1129)
Updating files:  90% (1017/1129)
Updating files:  91% (1028/1129)
Updating files:  92% (1039/1129)
Updating files:  93% (1050/1129)
Updating files:  94% (1062/1129)
Updating files:  95% (1073/1129)
Updating files:  96% (1084/1129)
Updating files:  97% (1096/1129)
Updating files:  98% (1107/1129)
Updating files:  99% (1118/1129)
Updating files: 100% (1129/1129)
Updating files: 100% (1129/1129), done.


Obtaining file:///C:/Users/user/Code/detection/mmdetection3d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: mmdet3d
    Found existing installation: mmdet3d 1.2.0
    Uninstalling mmdet3d-1.2.0:
      Successfully uninstalled mmdet3d-1.2.0
  Running setup.py develop for mmdet3d


## We will compare the case with and without pre-process

### Command Description

**!python demo/pcd_demo.py ../000000.bin ../pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py ../hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth --show**

**pcd_demo.py:** Script to run a demo of point cloud data.

**000000.bin:** Point cloud data to be inferred. Can be changed to other bin files.

**pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py:** configuration file for the model to be used.

**hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth:** checkpoint file containing weights for the learned model.

**--show:** Option to visualize and display inference results.



With this command, object detection is performed on the specified point cloud data and the results are displayed on the screen.

### This makes the preprocessed file an input

In [3]:
!python demo/pcd_demo.py ../filtered/filtered_points.bin ../pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py ../hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth --show


python: can't open file 'c:\Users\user\Code\detection\demo\pcd_demo.py': [Errno 2] No such file or directory


### This takes as input a file that has not been preprocessed.

In [ ]:
!python demo/pcd_demo.py ../000000.bin ../pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py ../hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth --show


Loads checkpoint by local backend from path: ../hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth


c:\users\user\code\presentation\mmdetection3d\mmdet3d\models\dense_heads\anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\mmengine\visualization\visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
